In [2]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# from sklearn.metrics import score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingRegressor
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [3]:
df = pd.read_csv('xlm.csv',parse_dates=True, index_col=('date'))
df.dropna(inplace=True)
df.rename(columns={'date':'Date','PriceUSD':'Close'}, inplace=True)
futures = df.tail()

df1 = df[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
futures1 = df1.tail()

df2 = df[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
futures2 = df2.tail()


In [4]:
df.Close.tail()

date
2019-11-03    0.068625
2019-11-04    0.074632
2019-11-05    0.082741
2019-11-06    0.077046
2019-11-07    0.074342
Name: Close, dtype: float64

In [5]:
#Model0
X=df[:-5].copy()
X.head()
y = df["Close"][5:].values.reshape(-1, 1)
y[:5]
#Model1
X1=df1[:-5].copy()
X1=X1[['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
y1 = df1["Close"][5:].values.reshape(-1, 1)
y1[:5]
#Model2
X2=df2[:-5].copy()
X2=X2[['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
y2 = df2["Close"][5:].values.reshape(-1, 1)
y2[:5]

array([[0.00215766],
       [0.00303475],
       [0.00285054],
       [0.00282279],
       [0.00365496]])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state=78)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=78)

In [7]:
# Scale the training data
scaler = StandardScaler()
scaler1=StandardScaler()
scaler2=StandardScaler()
#Model0
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
#model1
X_scaler1 = scaler1.fit(X_train1)
X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)
#model2
X_scaler2 = scaler2.fit(X_train2)
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

In [49]:
# Create GradientBoostingClassifier model
#Changed from .75 to .25 11/8
#on 11/8 .75 gave an accuracy score of .929
#model0

model = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.378,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model.score(
        X_train_scaled,
        y_train)))
print("Accuracy score (validation): {0:.3f}".format(
    model.score(
        X_test_scaled,
        y_test)))

Accuracy score (training): 1.000
Accuracy score (validation): 0.953


In [50]:
#model1 - [['FeeMedUSD','TxTfrValMedUSD','Close','IssTotUSD','VtyDayRet30d']]
# Create GradientBoostingClassifier model

# Choose learning rate
# learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
# for learning_rate in learning_rates:
#     model = GradientBoostingRegressor(
#         n_estimators=100,
#         learning_rate=learning_rate,
#         max_features=2,
#         max_depth=3,
#         random_state=0)
#     model.fit(X_train_scaled,y_train.ravel())
#     print("Learning rate: ", learning_rate)

#     # Score the model
#     print("Accuracy score (training): {0:.3f}".format(
#         model.score(
#             X_train_scaled,
#             y_train.ravel())))
#     print("Accuracy score (validation): {0:.3f}".format(
#         model.score(
#             X_test_scaled,
#             y_test.ravel())))
#     print()
    
model1 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.15,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model1.fit(X_train_scaled1,y_train1.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model1.score(
        X_train_scaled1,
        y_train1)))
print("Accuracy score (validation): {0:.3f}".format(
    model1.score(
        X_test_scaled1,
        y_test1)))

Accuracy score (training): 1.000
Accuracy score (validation): 0.919


In [51]:
#model2 - [['FeeMedUSD','TxTfrValMedUSD','IssTotUSD','VtyDayRet30d','TxTfrValMeanNtv']]
#Changed from .1 to .05 11/8
#on 11/8 .1 gave an accuracy score of .938
model2 = GradientBoostingRegressor(
    n_estimators=500,
    learning_rate=.05,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model2.fit(X_train_scaled2,y_train2.ravel())

# Score the model
print("Accuracy score (training): {0:.3f}".format(
    model2.score(
        X_train_scaled2,
        y_train2)))
print("Accuracy score (validation): {0:.3f}".format(
    model2.score(
        X_test_scaled2,
        y_test2)))

Accuracy score (training): 0.999
Accuracy score (validation): 0.765


In [52]:
# Make predictions
#model0
predictions = model.predict(X_test_scaled)
y_test = y_test.ravel()
actual = pd.Series(y_test)

#model1
predictions1 = model1.predict(X_test_scaled1)
y_test1 = y_test1.ravel()
actual1 = pd.Series(y_test1)
#model2
predictions2 = model2.predict(X_test_scaled2)
y_test2 = y_test2.ravel()
actual2 = pd.Series(y_test2)

In [53]:
pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual': actual})

,Model0,Model1,Model2,Actual
0,0.110437,0.119192,0.150021,0.076180
1,0.065465,0.064134,0.066073,0.058310
2,0.130732,0.114503,0.111421,0.133190
3,0.073603,0.062820,0.064071,0.077046
4,0.089162,0.098394,0.088061,0.083872
5,0.060695,0.061885,0.059287,0.061619
6,0.059913,0.061421,0.064182,0.060409
7,0.006220,0.002558,0.001621,0.003655
8,0.061115,0.048821,0.027579,0.020379
9,0.129611,0.116191,0.114591,0.121062


In [54]:
X.tail().shape


(5, 34)

In [55]:
X_future = X_scaler.transform(X.tail(5))
predictions = model.predict(X_future)

X_future1 = X_scaler1.transform(X1.tail(5))
predictions1 = model1.predict(X_future1)

X_future2 = X_scaler2.transform(X2.tail(5))
predictions2 = model2.predict(X_future2)

In [56]:
actual=futures['Close'].ravel()
dates=futures.reset_index()
dates=dates['date'].tail(5).ravel()
#Model1
actual1=futures1['Close'].ravel()
dates1=futures1.reset_index()
dates1=dates1['date'].tail(5).ravel()
#Model2
actual2=futures2['Close'].ravel()
dates2=futures2.reset_index()
dates2=dates2['date'].tail(5).ravel()

In [57]:
pd.DataFrame({'Model0':predictions,'Model1':predictions1,'Model2':predictions2,'Actual':actual, 'Date':dates})

,Model0,Model1,Model2,Actual,Date
0,0.073391,0.065203,0.073733,0.068625,2019-11-03
1,0.074840,0.073970,0.071384,0.074632,2019-11-04
2,0.082292,0.082389,0.077656,0.082741,2019-11-05
3,0.073603,0.062820,0.064071,0.077046,2019-11-06
4,0.073185,0.058978,0.064127,0.074342,2019-11-07


In [58]:
#Model0
real_future = X_scaler.transform(df.tail(5))
predictions = model.predict(real_future)
#Model1
real_future1 = X_scaler1.transform(df1.tail(5))
predictions1 = model1.predict(real_future1)
#Model2
real_future2 = X_scaler2.transform(df2.tail().drop('Close',axis=1))
predictions2 = model2.predict(real_future2)

Based on my predictions today, (Nov 7th)- I should sell my stock tomorrow (Nov 8th)


In [59]:
datelist = pd.date_range((pd.datetime.today()), periods=5).tolist()

# datelist = pd.date_range((pd.datetime.today()+timedelta(days=1)), periods=5).tolist()
future_dates=[]
for date in datelist:
    future_dates.append(date.strftime("%m-%d-%Y"))

In [60]:
final=pd.DataFrame({'Date':future_dates,'Model0':predictions,'Model1':predictions1,'Model2':predictions2})
final['Avg']=(final.Model0+final.Model1+final.Model2) / 3

In [61]:
final

,Date,Model0,Model1,Model2,Avg
0,11-08-2019,0.071268,0.063735,0.067217,0.067407
1,11-09-2019,0.083883,0.058907,0.049106,0.063966
2,11-10-2019,0.081183,0.076341,0.070021,0.075848
3,11-11-2019,0.079501,0.072958,0.063928,0.072129
4,11-12-2019,0.065949,0.055017,0.062906,0.061291


In [63]:
#BEST MODEL
#Model_0
final[['Date','Model0']]

,Date,Model0
0,11-08-2019,0.071268
1,11-09-2019,0.083883
2,11-10-2019,0.081183
3,11-11-2019,0.079501
4,11-12-2019,0.065949
